In [ ]:
import pandas as pd
import json 
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
# the correspodance between categories and amenities
with open('categories-amenities.json') as f:
    categories = json.load(f)

categories

### Nearest dataframe without home locations

In [ ]:
# original dataframe
df = pd.read_csv('../nearest_hundred_final.csv')
df['start'] = pd.to_datetime(df['start'])
df['end'] = pd.to_datetime(df['end'])
df['label'] = df['label'].apply(int)
df['label'] = df['label'].apply(str)
df['user'] = df['user'].apply(str)
df['category'] = df['category'].apply(str)
df.head()

In [ ]:
# find duration
df['duration'] = df['end'] - df['start']
# convert duration in hours
df['duration'] = df['duration'] / np.timedelta64(1, 's') / 60 / 60
df.head()

In [ ]:
new_df = df[["user", "start", "category", "duration" ]]
new_df = new_df.rename(columns={"start": "date"})
new_df['date'] = new_df['date'].dt.date
new_df

### Home locations dataframe

In [ ]:
home_df = pd.read_csv('../daily_home_hours.csv')
home_df = home_df.rename(columns={"start_date": "date", "duration_h":"duration"})
home_df['date'] = pd.to_datetime(home_df['date'])
home_df['user'] = home_df['user'].apply(str)
home_df['category'] = 'Residence'
home_df = home_df[["user", "date", "category", "duration" ]]
home_df

### Unite dataframes vertically

In [ ]:
final_df = new_df.append(home_df, ignore_index=True)
final_df

### Get matrix

In [ ]:
df1 = final_df.groupby(['user', 'date', 'category'])['duration'].sum().reset_index()

In [ ]:
df1 = df1.groupby(['user', 'category']).mean().reset_index()

In [ ]:
df1

In [ ]:
df1.pivot(index='user', columns='category', values='duration')

In [ ]:
def get_matrix(df):
    df = df.groupby(['user', 'date', 'category'])['duration'].sum().reset_index()
    df = df.groupby(['user', 'category']).mean().reset_index()
    df = df.pivot(index='user', columns='category', values='duration')
    return(df)

In [ ]:
get_matrix(final_df)